## Exploring the `fhirclient` package: creating a Patient, and POSTing it to a FHIR server
* install with 
    * `pip install fhirclient`
    * note: I modified `fhirclient/models/fhirabstractresource.py` so I can get the full response headers (see below), so copied the [`fhirclient`](https://github.com/smart-on-fhir/client-py/tree/master/fhirclient) package into my repo and edited it there. Doing this, I don't need to do the `pip install fhirclient`.
* versions
    * 3.2 (current at time of writing) is for STU3
    * 1.0.3 is for DSTU2
* github
    * https://github.com/smart-on-fhir/client-py
* Documentation
    * http://docs.smarthealthit.org/client-py/
    
There is an example in the documentation that shows how to create a Patient resource...

In [1]:
import fhirclient.models.patient as p
patient = p.Patient()

But the docs are sparse at describing the methods and attributes that are in it, at least for newbies. 

### Using `help()`
We can use `help()` to find out more about the `patient` object.

In [2]:
help(patient)

Help on Patient in module fhirclient.models.patient object:

class Patient(fhirclient.models.domainresource.DomainResource)
 |  Information about an individual or animal receiving health care services.
 |  
 |  Demographics and other administrative information about an individual or
 |  animal receiving care or other health-related services.
 |  
 |  Method resolution order:
 |      Patient
 |      fhirclient.models.domainresource.DomainResource
 |      fhirclient.models.resource.Resource
 |      fhirclient.models.fhirabstractresource.FHIRAbstractResource
 |      fhirclient.models.fhirabstractbase.FHIRAbstractBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, jsondict=None, strict=True)
 |      Initialize all valid properties.
 |      
 |      :raises: FHIRValidationError on validation errors, unless strict is False
 |      :param dict jsondict: A JSON dictionary to use for initialization
 |      :param bool strict: If True (the default), invalid variab

### The `.elementProperties()` method looks interesting

```
|  elementProperties(self)
|    Returns a list of tuples, one tuple for each property that should
|    be serialized, as: ("name", "json_name", type, is_list, "of_many", not_optional)
```

In [3]:
patient.elementProperties()

[('id', 'id', str, False, None, False),
 ('implicitRules', 'implicitRules', str, False, None, False),
 ('language', 'language', str, False, None, False),
 ('meta', 'meta', fhirclient.models.meta.Meta, False, None, False),
 ('contained',
  'contained',
  fhirclient.models.resource.Resource,
  True,
  None,
  False),
 ('extension',
  'extension',
  fhirclient.models.extension.Extension,
  True,
  None,
  False),
 ('modifierExtension',
  'modifierExtension',
  fhirclient.models.extension.Extension,
  True,
  None,
  False),
 ('text', 'text', fhirclient.models.narrative.Narrative, False, None, False),
 ('active', 'active', bool, False, None, False),
 ('address', 'address', fhirclient.models.address.Address, True, None, False),
 ('animal',
  'animal',
  fhirclient.models.patient.PatientAnimal,
  False,
  None,
  False),
 ('birthDate',
  'birthDate',
  fhirclient.models.fhirdate.FHIRDate,
  False,
  None,
  False),
 ('communication',
  'communication',
  fhirclient.models.patient.PatientComm

Just as `help()` told us, we see a list of tuples. 
<p>
Let's make it a little easier to read.

In [4]:
for eproperty in patient.elementProperties():
    print(eproperty)

('id', 'id', <class 'str'>, False, None, False)
('implicitRules', 'implicitRules', <class 'str'>, False, None, False)
('language', 'language', <class 'str'>, False, None, False)
('meta', 'meta', <class 'fhirclient.models.meta.Meta'>, False, None, False)
('contained', 'contained', <class 'fhirclient.models.resource.Resource'>, True, None, False)
('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('text', 'text', <class 'fhirclient.models.narrative.Narrative'>, False, None, False)
('active', 'active', <class 'bool'>, False, None, False)
('address', 'address', <class 'fhirclient.models.address.Address'>, True, None, False)
('animal', 'animal', <class 'fhirclient.models.patient.PatientAnimal'>, False, None, False)
('birthDate', 'birthDate', <class 'fhirclient.models.fhirdate.FHIRDate'>, False, None, False)
('communication', 'communicatio

Each tuple has six members. From the `help(patient)` result, we see what each member is:
```
|  elementProperties(self)
|    Returns a list of tuples, one tuple for each property that should
|    be serialized, as: ("name", "json_name", type, is_list, "of_many", not_optional)
```
The first three items in a tuple are easy to figure out. The last three required looking at the FHIR [Patient](http://build.fhir.org/patient.html) resource specification to confirm suspicions.
* `is_list` = Boolean that indicates if more than one member is allowed and therefore should be in a `list`, e.g., cardinality of `0..*`
* `of_many` = choice of data types (pick one), with prefix indicated, e.g, `deceasedBoolean` or `deceasedDateTime`
* `not_optional` = required, e.g., cardinality of `1..*`

How many of these patient properties are required?

In [5]:
required = []
for eproperty in patient.elementProperties():
    if eproperty[5] == True:
        required.append(eproperty)
len(required)

0

Nothing is actually required for this resource. Everything is optional.

How many can have more than one and need to be captured in a `list`?

In [6]:
islist = []
for eproperty in patient.elementProperties():
    if eproperty[3] == True:
        islist.append(eproperty)
len(islist)

12

What are they?

In [7]:
for eproperty in islist:
    print(eproperty)

('contained', 'contained', <class 'fhirclient.models.resource.Resource'>, True, None, False)
('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('address', 'address', <class 'fhirclient.models.address.Address'>, True, None, False)
('communication', 'communication', <class 'fhirclient.models.patient.PatientCommunication'>, True, None, False)
('contact', 'contact', <class 'fhirclient.models.patient.PatientContact'>, True, None, False)
('generalPractitioner', 'generalPractitioner', <class 'fhirclient.models.fhirreference.FHIRReference'>, True, None, False)
('identifier', 'identifier', <class 'fhirclient.models.identifier.Identifier'>, True, None, False)
('link', 'link', <class 'fhirclient.models.patient.PatientLink'>, True, None, False)
('name', 'name', <class 'fhirclient.models.humanname.HumanName'>, True, None, False)
('photo', 'photo

Those 12 properties should be captured in a `list`.
<p>
Which, if any, of the patient properties are `of_many`?

In [8]:
for eproperty in patient.elementProperties():
    if eproperty[4] != None:
        print(eproperty)

('deceasedBoolean', 'deceasedBoolean', <class 'bool'>, False, 'deceased', False)
('deceasedDateTime', 'deceasedDateTime', <class 'fhirclient.models.fhirdate.FHIRDate'>, False, 'deceased', False)
('multipleBirthBoolean', 'multipleBirthBoolean', <class 'bool'>, False, 'multipleBirth', False)
('multipleBirthInteger', 'multipleBirthInteger', <class 'int'>, False, 'multipleBirth', False)


<div class="alert alert-block alert-info">A word about choice of data types. From http://build.fhir.org/formats.html#choice,
<p>
<i>Elements that have a choice of data type cannot repeat - they must have a maximum cardinality of 1. When constructing an instance of an element with a choice of types, the authoring system must create a single element with a data type chosen from among the list of permitted data types.</i>
</div>

### The `.as_json()` method
Another method for the patient object is `.as_json()` which was inherited from <br>
`fhirclient.models.fhirabstractresource.FHIRAbstractResource`<p>
from `help(patient)`
```
 |   as_json(self)
 |      Serializes to JSON by inspecting `elementProperties()` and creating
 |      a JSON dictionary of all registered properties. Checks:
 |      
 |      - whether required properties are not None (and lists not empty)
 |      - whether not-None properties are of the correct type
 |      
 |      :raises: FHIRValidationError if properties have the wrong type or if
 |          required properties are empty
 |      :returns: A validated dict object that can be JSON serialized
```
    
Let's try it.

In [9]:
patient.as_json()

{'resourceType': 'Patient'}

When patient was created, it automatically filled in the `resourceType`, which interestingly isn't one of the properties returned by `patient.elementProperties`

We can pretty print this with `json.dumps()`

In [10]:
import json
print(json.dumps(patient.as_json(), indent=4))

{
    "resourceType": "Patient"
}


### Add a name to Patient
<p>
We see from `patient.elementsProperties()` that `name` has a type of `<class 'fhirclient.models.humanname.HumanName'>`
<p>
We'll create a `HumanName` object to hold this, and find out the properties it has.

In [11]:
import fhirclient.models.humanname as hn
name = hn.HumanName()
for eproperty in name.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('family', 'family', <class 'str'>, False, None, False)
('given', 'given', <class 'str'>, True, None, False)
('period', 'period', <class 'fhirclient.models.period.Period'>, False, None, False)
('prefix', 'prefix', <class 'str'>, True, None, False)
('suffix', 'suffix', <class 'str'>, True, None, False)
('text', 'text', <class 'str'>, False, None, False)
('use', 'use', <class 'str'>, False, None, False)


#### Let's build a name
Since `given` has `is_list` set to `True`, we need to put it in a `list` or we'll get a `FHIRValidationError`. This is so we can have additional names as part of the full name (e.g., middle names). Since we can, we'll add a middle name. 

In [12]:
name.family = 'Storm'
name.given = ['Jonathon', 'Lowell']
name.text = 'Jonathon Lowell Storm'

We can now define how the name is used. I haven't figured out if the valueset for `use` can be found in the models, but reading the specification for [`HumanName`](http://hl7.org/implement/standards/fhir/datatypes.html#HumanName), I see that `use` must be from http://hl7.org/implement/standards/fhir/valueset-name-use.html. We'll make it for `official` use. If we mistype this, a `FHIRValidationError` won't pop up here, but the FHIR server might when we POST it.

In [13]:
name.use = 'official'
print(json.dumps(name.as_json(), indent=4))

{
    "family": "Storm",
    "given": [
        "Jonathon",
        "Lowell"
    ],
    "text": "Jonathon Lowell Storm",
    "use": "official"
}


Now add `name` to `patient`. Again, since `name` has `is_list` set to true, we need to put it in a list.

In [14]:
patient.name = [name]
print(json.dumps(patient.as_json(), indent=4))

{
    "name": [
        {
            "family": "Storm",
            "given": [
                "Jonathon",
                "Lowell"
            ],
            "text": "Jonathon Lowell Storm",
            "use": "official"
        }
    ],
    "resourceType": "Patient"
}


Since we can have more than one name, let's add a couple of nicknames.

In [15]:
name2 = hn.HumanName()
name2.given = ['Johnny']
name2.family = 'Storm'
name2.use = 'nickname'
name3 = hn.HumanName()
name3.text = 'Human Torch'
name3.use = 'nickname'
patient.name.extend([name2, name3])
print(json.dumps(patient.as_json(), indent=4))

{
    "name": [
        {
            "family": "Storm",
            "given": [
                "Jonathon",
                "Lowell"
            ],
            "text": "Jonathon Lowell Storm",
            "use": "official"
        },
        {
            "family": "Storm",
            "given": [
                "Johnny"
            ],
            "use": "nickname"
        },
        {
            "text": "Human Torch",
            "use": "nickname"
        }
    ],
    "resourceType": "Patient"
}


### POSTing a Patient
Now that we built a Patient resource, we can try to POST it to an open FHIR server. We'll use the examples  are in github repo README to guide us, and use http://fhirtest.b12.org/baseDstu3 for the server. Although the example didn't have a POST example, we see a `create()` method described in `help(patient)`
```
 |  create(self, server)
 |      Attempt to create the receiver on the given server, using a POST
 |      command.
 |      
 |      :param FHIRServer server: The server to create the receiver on
 |      :returns: None or the response JSON on success                                     
``` 
We can create a smart object with server info as done in the example, and see what's inside it.

In [16]:
from fhirclient import client
settings = {
        'app_id': 'my_web_app',
        'api_base': 'http://fhirtest.b12x.org/baseDstu3'
    }
smart = client.FHIRClient(settings=settings)
smart.__dict__

{'app_id': 'my_web_app',
 'app_secret': None,
 'server': <server.FHIRServer at 0x109206048>,
 'scope': 'user/*.* patient/*.read openid profile',
 'redirect': None,
 'launch_token': None,
 'launch_context': None,
 'wants_patient': True,
 'patient_id': None,
 '_patient': None,
 '_save_func': <function fhirclient.client.FHIRClient.<lambda>(x)>}

In [17]:
help(smart)

Help on FHIRClient in module fhirclient.client object:

class FHIRClient(builtins.object)
 |  Instances of this class handle authorizing and talking to SMART on FHIR
 |  servers.
 |  
 |  The settings dictionary supports:
 |  
 |      - `app_id`*: Your app/client-id, e.g. 'my_web_app'
 |      - `app_secret`*: Your app/client-secret
 |      - `api_base`*: The FHIR service to connect to, e.g. 'https://fhir-api-dstu2.smarthealthit.org'
 |      - `redirect_uri`: The callback/redirect URL for your app, e.g. 'http://localhost:8000/fhir-app/' when testing locally
 |      - `patient_id`: The patient id against which to operate, if already known
 |      - `scope`: Space-separated list of scopes to request, if other than default
 |      - `launch_token`: The launch token
 |  
 |  Methods defined here:
 |  
 |  __init__(self, settings=None, state=None, save_func=<function FHIRClient.<lambda> at 0x109204048>)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  from_stat

We'll use `smart.server` as the server.

#### Before we POST, a note about the `response`...
As it is distributed, `patient.create()` returns json formatted  `Content`. I wanted access to the full HTTP header response, so I edited the `create`, `update`, and `delete` methods in `fhirclient/models/fhirabstractresource.py` file from (suggested by Pascal Pfiffner in this [chat.fhir.org thread](https://chat.fhir.org/#narrow/stream/python/subject/HTTP.20response.20header.20and.20location)):

from
```python    
    ret = srv.post_json(self.relativeBase(), self.as_json())
    if len(ret.text) > 0:
        return ret.json()
```  
to     
```python
    ret = srv.post_json(self.relativeBase(), self.as_json())
    if len(ret.text) > 0:
        # return the full requests.Response object
        return ret
        # return ret.json()
```

Now try POSTing the `patient` to the server using the `.create()` method

In [18]:
response = patient.create(smart.server)
type(response)

requests.models.Response

Mimic previous behaviour of `response` with `response.json()`

In [19]:
print(json.dumps(response.json(), indent=4))

{
    "resourceType": "Patient",
    "id": "4406",
    "meta": {
        "versionId": "1",
        "lastUpdated": "2019-01-22T03:31:06.498+00:00"
    },
    "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Jonathon Lowell <b>STORM </b></div><table class=\"hapiPropertyTable\"><tbody></tbody></table></div>"
    },
    "name": [
        {
            "use": "official",
            "text": "Jonathon Lowell Storm",
            "family": "Storm",
            "given": [
                "Jonathon",
                "Lowell"
            ]
        },
        {
            "use": "nickname",
            "family": "Storm",
            "given": [
                "Johnny"
            ]
        },
        {
            "use": "nickname",
            "text": "Human Torch"
        }
    ]
}


To see all the response headers...

In [20]:
print(response.headers)

{'Content-Encoding': 'gzip', 'Content-Location': 'http://fhirtest.b12x.org/r3/Patient/4406/_history/1', 'Content-Type': 'application/fhir+json;charset=UTF-8', 'Date': 'Tue, 22 Jan 2019 03:31:06 GMT', 'ETag': 'W/"1"', 'Last-Modified': 'Tue, 22 Jan 2019 03:31:06 GMT', 'Location': 'http://fhirtest.b12x.org/r3/Patient/4406/_history/1', 'Server': 'Apache-Coyote/1.1', 'X-FHIR-Request-Validation': '{"resourceType":"OperationOutcome","text":{"status":"generated","div":"<div xmlns=\\"http://www.w3.org/1999/xhtml\\"><h1>Operation Outcome</h1><table border=\\"0\\"><tr><td style=\\"font-weight: bold;\\"...', 'X-Powered-By': 'HAPI FHIR 3.6.0 REST Server (FHIR Server; FHIR 3.0.1/DSTU3)', 'transfer-encoding': 'chunked', 'Connection': 'keep-alive'}


Make it easier to read...

In [21]:
for header in response.headers:
    print('{0}: {1}'.format(header, response.headers[header]))

Content-Encoding: gzip
Content-Location: http://fhirtest.b12x.org/r3/Patient/4406/_history/1
Content-Type: application/fhir+json;charset=UTF-8
Date: Tue, 22 Jan 2019 03:31:06 GMT
ETag: W/"1"
Last-Modified: Tue, 22 Jan 2019 03:31:06 GMT
Location: http://fhirtest.b12x.org/r3/Patient/4406/_history/1
Server: Apache-Coyote/1.1
X-FHIR-Request-Validation: {"resourceType":"OperationOutcome","text":{"status":"generated","div":"<div xmlns=\"http://www.w3.org/1999/xhtml\"><h1>Operation Outcome</h1><table border=\"0\"><tr><td style=\"font-weight: bold;\"...
X-Powered-By: HAPI FHIR 3.6.0 REST Server (FHIR Server; FHIR 3.0.1/DSTU3)
transfer-encoding: chunked
Connection: keep-alive


So, to see the `Location` of the resource that was created, I can...

In [22]:
print(response.headers['Location'])

http://fhirtest.b12x.org/r3/Patient/4406/_history/1


Next up, creating a `Specimen` resource that references the `Patient`.